<a href="https://colab.research.google.com/github/RashaAlshawi/EFPN/blob/main/DataImbalance/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#CHANGE WITH YOUR DATA
X_P=np.load('/content/drive/MyDrive/VideoFrames/PositiveImageNew.npy')
y_P=np.load('/content/drive/MyDrive/VideoFrames/PositivemaskNew.npy')

In [ ]:
X_P, y_P = shuffle(X_P, y_P, random_state=10)

In [ ]:
samples_train = int(0.80 * len(X_P))
samples_val = int(0.20 * len(y_P))

X_trainAndVal = X_datas[:samples_train]
y_trainAndVal = y_datas[:samples_train]

X_test = X_datas[samples_train:samples_train + samples_val]
y_test = y_datas[samples_train:samples_train + samples_val]

In [ ]:
def augment_data(images, labels):
    augmented_images = []
    augmented_labels = []

    # Ensure images are in uint8 format and channels are in the correct order
    images = np.uint8(images)
    labels = np.uint8(labels)

    for image, label in zip(images, labels):
        # Horizontal Flip
        augmented_image = cv2.flip(image, 1)  # 1 for horizontal flip
        augmented_label = cv2.flip(label, 1)  # 1 for horizontal flip
        augmented_images.append(augmented_image)
        augmented_labels.append(augmented_label)

        # Color Jittering
        brightness = np.random.uniform(0.5, 1.5)  # Random brightness factor
        contrast = np.random.uniform(0.5, 1.5)    # Random contrast factor
        saturation = np.random.uniform(0.5, 1.5)  # Random saturation factor
        hue = np.random.uniform(-0.1, 0.1)       # Random hue shift
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_image = np.array(hsv_image, dtype=np.float64)
        hsv_image[:, :, 2] *= brightness
        hsv_image[:, :, 1] *= contrast
        hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1], 0, 255)
        hsv_image[:, :, 1] *= saturation
        hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1], 0, 255)
        hsv_image[:, :, 0] += hue * 180
        hsv_image[:, :, 0] = np.clip(hsv_image[:, :, 0], 0, 180)
        image = np.array(hsv_image, dtype=np.uint8)
        augmented_images.append(image)
        augmented_labels.append(label)  # No change to label for color jittering
             # Shearing
        shear_factor = 0.2  # Shear factor
        shear_matrix = np.array([[1, shear_factor, 0], [0, 1, 0]])
        sheared_image = cv2.warpAffine(image, shear_matrix, (image.shape[1], image.shape[0]))
        sheared_label = cv2.warpAffine(label, shear_matrix, (label.shape[1], label.shape[0]))
        augmented_images.append(sheared_image)
        augmented_labels.append(sheared_label)


    # Rotate
        rotation_angle1 = 50  # in degrees
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), rotation_angle1, 1)
        rotated_image1 = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        rotated_label1= cv2.warpAffine(label, rotation_matrix, (label.shape[1], label.shape[0]))
        augmented_images.append(rotated_image1)
        augmented_labels.append(rotated_label1)


        # Random Crop
        crop_height, crop_width = 100, 100  # Adjust crop size as needed
        y_start = np.random.randint(0, image.shape[0] - crop_height)
        x_start = np.random.randint(0, image.shape[1] - crop_width)
        cropped_image = image[y_start:y_start+crop_height, x_start:x_start+crop_width]
        cropped_label = label[y_start:y_start+crop_height, x_start:x_start+crop_width]
        cropped_image = cv2.resize(cropped_image, (image.shape[1], image.shape[0]))  # Resize cropped image to match original size
        cropped_label = cv2.resize(cropped_label, (label.shape[1], label.shape[0]))  # Resize cropped label to match original size
        augmented_images.append(cropped_image)
        augmented_labels.append(cropped_label)


    return np.array(augmented_images), np.array(augmented_labels)

In [ ]:
augmented_images, augmented_labels = augment_data(X_trainAndVal, y_trainAndVal)

In [ ]:
# CHANGE WITH YOUR PATH
path='/content/drive/MyDrive/VideoFrames/DS/'

After applying data augmention, compine the original samples with the augmented once

In [ ]:
Xall=np.concatenate((X_trainAndVal,augmented_images))
yall=np.concatenate((y_trainAndVal,augmented_labels))

In [ ]:
# Save training and Validating images
np.save(path+'Xall_fortrainingAndValidating',Xall)
np.save(path+'yall_fortrainingAndValidating',yall)

In [ ]:
# Save testing
np.save(path+'X_test',X_test)
np.save(path+'X_test',X_test)